In [2]:
using StatsBase
using Combinatorics
using Plots

include("jl/omega.jl")
include("jl/HSBM.jl")
include("jl/cut.jl")

# parameters
# parameters

n = 100
Z = rand(1:2, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

# defining group intensity function Ω
μ = mean(ϑ)

ω(p,α) = (10 .*μ*sum(p))^(-sum(p))*prod(p.^α)^(1/(sum(p)*α))
α0 = 1

kmax = 3

Ω = buildΩ(ω, α0, kmax)

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1260


Ω (generic function with 1 method)

In [4]:
## Sample
H = sampleSBM(Z, ϑ, Ω; α=α0, kmax=kmax, kmin = 1)
countEdges(H)

92

In [5]:
## Create a random clustering with k clusters
include("jl/cut.jl")
k = 10
c = rand(1:k,n)
term1 = first_term_eval(H,c,Ω;α=α0)

-887.5596733815513

In [ ]:
@time T1 = first_term_eval(H,c,Ω;α=α0)
@time Hyp, w = hyperedge_formatting(H)
@time T2 = first_term_v2(Hyp,w,c,Ω;α=α0)

T1 ≈ T2

  0.000428 seconds (1.85 k allocations: 150.609 KiB)
  0.178125 seconds (167.28 k allocations: 8.687 MiB)
  0.044953 seconds (31.02 k allocations: 1.642 MiB)


The CutDiff function computes the change in objective score when node I is moved to cluster J. The code is still a little sloppy, but this is a little faster and a step in the right direction over a naive approach. One thing that is clear to me after implementing this is that we will need a new way to store hyperedge. Given a query node, we need to quickly be able to access all hyperedges that node is in.

In [6]:
include("jl/cut.jl")
node2edges = EdgeMap(H)
I = rand(1:n)
J = c[I] + 1
@time a = NaiveCutDiff(H,c,I,J,Ω)
@time b = CutDiff(Hyp,w,node2edges,c,I,J,Ω)
@show b, a
a ≈ b

  0.320767 seconds (1.81 M allocations: 122.384 MiB, 5.77% gc time)
  0.074287 seconds (89.83 k allocations: 5.866 MiB, 10.51% gc time)
(b, a) = (-23.88977823429559, -23.889778234646656)


true